In [ ]:
# Install necessary libraries
!pip install -q -U google-generativeai librosa streamlit pillow pandas kagglehub

import kagglehub
import os

# Download Datasets
audio_path = kagglehub.dataset_download("vbookshelf/respiratory-sound-database")
xray_path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")



print(f"Audio Data: {audio_path}")
print(f"X-Ray Data: {xray_path}")

/home/ibadat/miniconda3/envs/Hackathon/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 3.69G/3.69G [03:49<00:00, 17.3MB/s] 

Extracting files...


100%|██████████| 2.29G/2.29G [03:09<00:00, 13.0MB/s]

Extracting files...


Audio Data: /home/ibadat/.cache/kagglehub/datasets/vbookshelf/respiratory-sound-database/versions/2
X-Ray Data: /home/ibadat/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2


In [2]:
import pandas as pd
import random
import os # Added import for os module

def get_random_case(audio_root, xray_root):
    # 1. Pick an X-ray (Pneumonia or Normal)
    xray_folder = os.path.join(xray_root, 'chest_xray', 'test', 'PNEUMONIA')
    random_xray = os.path.join(xray_folder, random.choice(os.listdir(xray_folder)))

    # 2. Pick an Audio file (Respiratory sounds)
    # Corrected path based on debug prints, accounting for nested directories
    audio_folder = os.path.join(audio_root, 'respiratory_sound_database', 'Respiratory_Sound_Database', 'audio_and_txt_files')

    # Filter for .wav files
    wav_files = [f for f in os.listdir(audio_folder) if f.endswith('.wav')]
    random_audio = os.path.join(audio_folder, random.choice(wav_files))

    return random_xray, random_audio

# Test the generator
x_path, a_path = get_random_case(audio_path, xray_path)
print(f"Selected X-Ray: {x_path}\nSelected Audio: {a_path}")

Selected X-Ray: /home/ibadat/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/test/PNEUMONIA/person94_bacteria_458.jpeg
Selected Audio: /home/ibadat/.cache/kagglehub/datasets/vbookshelf/respiratory-sound-database/versions/2/respiratory_sound_database/Respiratory_Sound_Database/audio_and_txt_files/135_2b3_Tc_mc_LittC2SE.wav


In [3]:
pip install -U google-genai

Note: you may need to restart the kernel to use updated packages.


In [4]:
from google import genai
from google.genai import types
import PIL.Image

# 1. Initialize the Client
# Replace 'YOUR_API_KEY' with your actual key
client = genai.Client(api_key="YOUR_GEMINI_API_KEY_HERE")

def analyze_with_gemini_v3(xray_path, audio_path, vitals_data):
    # Load the image
    raw_image = PIL.Image.open(xray_path)

    # Load the audio (The new SDK handles bytes/files directly)
    with open(audio_path, "rb") as f:
        audio_bytes = f.read()

    # Define the System Instruction (The 'Brain' configuration)
    sys_instruct = "You are a Senior Medical Consultant. Analyze the provided X-ray, lung audio, and vitals to provide a differential diagnosis."

    # 2. Generate Content using the NEW Client syntax
    # Note: We use 'gemini-2.0-flash' or 'gemini-1.5-pro' depending on availability
    response = client.models.generate_content(
        model="gemini-2.0-flash", # Use the latest version available to you
        config=types.GenerateContentConfig(
            system_instruction=sys_instruct,
            temperature=0.2, # Lower temperature for medical accuracy
        ),
        contents=[
            f"Patient Vitals: {vitals_data}",
            raw_image,
            types.Part.from_bytes(data=audio_bytes, mime_type="audio/wav"),
            "Perform a multimodal cross-correlation. Does the audio match the image findings?"
        ]
    )

    return response.text

# --- Example Usage ---
vitals = {"Age": 65, "SpO2": "91%", "Pulse": 105}
# result = analyze_with_gemini_v3(x_path, a_path, vitals)
# print(result)

In [5]:
import streamlit as st
from PIL import Image

# UI Configuration
st.set_page_config(page_title="VITAL-LINK | Medical AI", layout="wide")

st.title("🏥 Project VITAL-LINK")
st.markdown("### Multimodal Clinical Reasoning Dashboard")
st.divider()

# Sidebar for Inputs
with st.sidebar:
    st.header("Patient Intake")
    age = st.slider("Patient Age", 0, 100, 45)
    gender = st.selectbox("Gender", ["Male", "Female", "Other"])
    temp = st.number_input("Temperature (°C)", value=37.5)
    spo2 = st.slider("SpO2 (%)", 80, 100, 95)

    st.divider()
    uploaded_xray = st.file_uploader("Upload Chest X-Ray", type=['jpg', 'png'])
    uploaded_audio = st.file_uploader("Upload Lung Sounds", type=['wav', 'mp3'])

# Main Panel
col1, col2 = st.columns([1, 1])

with col1:
    st.subheader("Input Visualizations")
    if uploaded_xray:
        st.image(uploaded_xray, caption="Chest X-Ray", use_column_width=True)
    if uploaded_audio:
        st.audio(uploaded_audio)
        st.caption("Auscultation Recording")

with col2:
    st.subheader("AI Clinical Reasoning")
    if st.button("Run Multi-Modal Diagnostic"):
        if uploaded_xray and uploaded_audio:
            with st.spinner("Gemini 3 is correlating medical data..."):
                # Call the backend function here
                # For demo purposes, we simulate the output
                vitals = {"Age": age, "SpO2": spo2, "Temp": temp}
                # result = analyze_patient_case(uploaded_xray, uploaded_audio, vitals)
                st.info("Analysis Complete")
                st.markdown("""
                **Findings:**
                - **Radiology:** Consolidation noted in the right lower lobe.
                - **Auscultation:** Late inspiratory crackles detected in corresponding area.
                - **Correlation:** High alignment between visual and auditory findings.

                **Differential Diagnosis:**
                1. Community-Acquired Pneumonia (85% Confidence)
                2. Bronchitis (10% Confidence)

                **Reasoning:** The combination of febrile state (37.5C), localized crackles, and X-ray opacities strongly indicates bacterial pneumonia.
                """)
        else:
            st.error("Please upload both X-Ray and Audio files.")

# Professional Footer
st.divider()
st.caption("VITAL-LINK is an AI research prototype built for the Gemini 3 Hackathon. Not for clinical use.")

2026-01-26 20:29:29.489 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-26 20:29:29.489 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-26 20:29:29.572 
  command:

    streamlit run /home/ibadat/miniconda3/envs/Hackathon/lib/python3.14/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-01-26 20:29:29.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-26 20:29:29.575 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-26 20:29:29.576 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-26 20:29:29.577 Thread 'MainThread': missing ScriptRunContext! This warning

DeltaGenerator()